# Groups

In [1]:
import pandas as pd
import numpy as np

#### Tabla ***region***

In [2]:
groups = ['Africa','Asia','Central America','Europe','North America','Oceania','South America', 'World','G20 (Ember)']

In [3]:
region = pd.DataFrame(groups, columns=['region']).sort_values(by='region').reset_index().rename(columns={'index':'idRegion'})

In [4]:
region.to_csv('../Modelo_ER/region.csv', index=False)

#### Tabla ***emission_region***


In [5]:
energy_consumption = pd.read_csv("https://github.com/Wolfgang93/emissionsCO2/blob/main/rawDataset/owid-energy-consumption-source.csv?raw=true")
owid_co2_data = pd.read_csv('../rawDataset/owid-co2-data.csv')
countries = pd.read_csv('../Modelo_ER/country.csv').country.values
regions = pd.read_csv('../Modelo_ER/region.csv').region.values

##### Procesamiento energy_consumption

In [6]:
quitar_columnas = ['biofuel_cons_change_pct','biofuel_cons_change_twh','biofuel_cons_per_capita','biofuel_consumption','biofuel_elec_per_capita','biofuel_electricity','biofuel_share_elec','biofuel_share_energy','coal_cons_change_pct','coal_cons_change_twh','coal_cons_per_capita','coal_prod_change_pct','coal_prod_change_twh','coal_prod_per_capita','coal_production','coal_share_elec','coal_share_energy','energy_cons_change_pct','energy_cons_change_twh','fossil_cons_change_pct','fossil_cons_change_twh','fossil_elec_per_capita','fossil_electricity','fossil_energy_per_capita','fossil_fuel_consumption','fossil_share_elec','fossil_share_energy','gas_cons_change_pct','gas_cons_change_twh','gas_prod_change_pct','gas_prod_change_twh','gas_prod_per_capita','gas_production','gas_share_elec','gas_share_energy','hydro_cons_change_pct','hydro_cons_change_twh','hydro_share_elec','hydro_share_energy','low_carbon_cons_change_pct','low_carbon_cons_change_twh','low_carbon_consumption','low_carbon_elec_per_capita','low_carbon_electricity','low_carbon_energy_per_capita','low_carbon_share_elec','low_carbon_share_energy','net_elec_imports','net_elec_imports_share_demand','nuclear_cons_change_pct','nuclear_cons_change_twh','nuclear_share_elec','nuclear_share_energy','oil_cons_change_pct','oil_cons_change_twh','oil_prod_change_pct','oil_prod_change_twh','oil_prod_per_capita','oil_production','oil_share_elec','oil_share_energy','other_renewable_exc_biofuel_electricity','other_renewables_cons_change_pct','other_renewables_cons_change_twh','other_renewables_elec_per_capita_exc_biofuel','other_renewables_share_elec','other_renewables_share_elec_exc_biofuel','other_renewables_share_energy','renewables_cons_change_pct', 'renewables_cons_change_twh','renewables_share_elec','renewables_share_energy','solar_cons_change_pct','solar_cons_change_twh','solar_share_elec','solar_share_energy','wind_cons_change_pct','wind_cons_change_twh','wind_share_elec','wind_share_energy','energy_per_gdp', 'gdp', 'primary_energy_consumption']

In [7]:
energy_consumption = energy_consumption.drop(quitar_columnas, axis=1)
energy_consumption.drop_duplicates(inplace=True)
energy_consumption = energy_consumption.fillna(0)
energy_consumption = energy_consumption[energy_consumption['year'] >= 1980]
energy_consumption = energy_consumption.replace(',', ' ', regex=True)
energy_consumption['population'] = energy_consumption['population'].astype(int)
energy_consumption = energy_consumption.replace('Central America (BP)','Central America')

In [8]:
energy_consumption_region = energy_consumption[energy_consumption.country.isin(regions)]

In [9]:
energy_consumption_region = energy_consumption_region[['country', 'year', 'population', 'greenhouse_gas_emissions']]

In [10]:
energy_consumption_region = energy_consumption_region.rename(columns={'greenhouse_gas_emissions':'co2emissions', 'country':'region'})

##### Procesamiento owid_co2_data

In [11]:
cols = ['country', 'year', 'co2_per_capita']
owid_co2_data = owid_co2_data[cols]
owid_co2_data = owid_co2_data[owid_co2_data['year'] >= 1980]
owid_co2_data.columns = ['region', 'year', 'co2PerCapita']
owid_co2_data = owid_co2_data.replace('Micronesia (country)','Micronesia')
owid_co2_data = owid_co2_data.replace('Timor','Timor-Leste')
owid_co2_data = owid_co2_data.replace('Guinea-Bissau','Guinea Bissau')

In [12]:
owid_co2_regions = owid_co2_data[owid_co2_data['region'].isin(regions)]

##### Merge de ambos DF finales de los procesamientos

In [13]:
emission_region = energy_consumption_region.merge(owid_co2_regions, on=['region', 'year'], how = 'left').fillna(0).sort_values(['region', 'year'])

In [14]:
emission_region

,region,year,population,co2emissions,co2PerCapita
0,Africa,1980,476386229,0.00,1.137
1,Africa,1981,490003902,0.00,1.132
2,Africa,1982,504034174,0.00,1.146
3,Africa,1983,518479760,0.00,1.159
4,Africa,1984,533345153,0.00,1.230
...,...,...,...,...,...
353,World,2017,-2147483648,11467.42,4.760
354,World,2018,-2147483648,11842.78,4.802
355,World,2019,-2147483648,11766.67,4.758
356,World,2020,-2147483648,11343.30,4.465


In [15]:
emission_region.to_csv('../Modelo_ER/emission_region.csv', index=False)

#### Tabla ***source_region_electricity***


##### Procesamiento

In [16]:
energy_consumption = energy_consumption.rename(columns={'other_renewable_consumption':'other_renewables_consumption','other_renewable_electricity':'other_renewables_electricity'})

In [17]:
energy_consumption['coal_energy_per_capita'] = np.zeros(len(energy_consumption))

In [18]:
energy_consumption = energy_consumption.rename(columns={'other_renewable_consumption':'other_renewables_consumption','other_renewable_electricity':'other_renewables_electricity'})

In [19]:
source_region_electricity = energy_consumption.drop(['iso_code','population','carbon_intensity_elec','electricity_demand','electricity_generation','energy_per_capita','greenhouse_gas_emissions'], axis=1)

In [20]:
source_region_electricity = source_region_electricity[source_region_electricity.country.isin(regions)]

In [21]:
source_region_electricity

,country,year,coal_consumption,coal_elec_per_capita,coal_electricity,gas_consumption,gas_elec_per_capita,gas_electricity,gas_energy_per_capita,hydro_consumption,...,renewables_energy_per_capita,solar_consumption,solar_elec_per_capita,solar_electricity,solar_energy_per_capita,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,coal_energy_per_capita
201,Africa,1980,543.557,0.000,0.000,186.914,0.000,0.000,392.358,142.857,...,301.264,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
202,Africa,1981,635.799,0.000,0.000,228.491,0.000,0.000,466.305,148.420,...,304.315,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
203,Africa,1982,701.176,0.000,0.000,246.734,0.000,0.000,489.517,148.459,...,296.037,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
204,Africa,1983,713.246,0.000,0.000,271.275,0.000,0.000,523.213,133.795,...,259.469,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
205,Africa,1984,762.898,0.000,0.000,259.903,0.000,0.000,487.307,131.556,...,248.063,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21961,World,2017,43751.676,1287.289,9716.677,36529.223,789.315,5957.887,4839.479,10831.405,...,2368.127,1185.515,59.016,445.466,157.060,3034.917,151.082,1140.393,402.073,0.0
21962,World,2018,44315.875,1323.230,10097.678,38356.297,813.338,6206.648,5026.325,11067.518,...,2489.008,1524.550,75.511,576.231,199.782,3360.019,166.422,1269.979,440.307,0.0
21963,World,2019,43699.887,1278.688,9863.103,39062.945,832.442,6421.006,5064.258,11153.792,...,2588.879,1855.592,91.263,703.950,240.566,3744.515,184.164,1420.544,485.452,0.0
21964,World,2020,41963.961,1210.971,9439.266,38455.727,817.424,6371.650,4933.518,11413.827,...,2704.402,2222.443,108.563,846.229,285.119,4192.682,204.807,1596.428,537.883,0.0


##### Pivot de las columnas

In [22]:
sources = ['coal', 'gas', 'hydro', 'nuclear', 'oil', 'other_renewables', 'renewables', 'solar', 'wind']
cols = ['region','year', 'source', 'consumptionElectricity', 'productionElectricity', 'energyPerCapita', 'elecPerCapita']

In [23]:
base_df = pd.DataFrame(columns = cols)

In [24]:
for n in sources:
    agg = source_region_electricity[['country', 'year', f"{n}_consumption", f"{n}_electricity", f"{n}_energy_per_capita", f"{n}_elec_per_capita"]]
    agg.columns = ['region','year','consumptionElectricity', 'productionElectricity', 'energyPerCapita', 'elecPerCapita']
    agg.insert(2, 'source', n)
    base_df = base_df.append(agg)

C:\Users\Ale\AppData\Local\Temp\ipykernel_9704\1851924376.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_df = base_df.append(agg)
C:\Users\Ale\AppData\Local\Temp\ipykernel_9704\1851924376.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_df = base_df.append(agg)
C:\Users\Ale\AppData\Local\Temp\ipykernel_9704\1851924376.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_df = base_df.append(agg)
C:\Users\Ale\AppData\Local\Temp\ipykernel_9704\1851924376.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_df = base_df.append(agg)
C:\Users\Ale\AppData\Local\Temp\ipykernel_9704\1851924376.py:5: FutureWarning: The f

In [25]:
base_df.drop_duplicates(subset=['region', 'year', 'source'], inplace=True)
base_df.sort_values(by=['region','year','source'], inplace=True)

In [26]:
base_df

,region,year,source,consumptionElectricity,productionElectricity,energyPerCapita,elecPerCapita
201,Africa,1980,coal,543.557,0.000,0.000,0.000
201,Africa,1980,gas,186.914,0.000,392.358,0.000
201,Africa,1980,hydro,142.857,48.312,299.877,101.413
201,Africa,1980,nuclear,0.000,0.000,0.000,0.000
201,Africa,1980,oil,817.239,0.000,1715.496,0.000
...,...,...,...,...,...,...,...
21965,World,2021,oil,51170.477,720.286,6497.875,91.465
21965,World,2021,other_renewables,2373.099,762.783,301.348,96.862
21965,World,2021,renewables,22270.062,7931.052,2827.960,1007.123
21965,World,2021,solar,2701.722,1032.501,343.078,131.112


In [27]:
base_df.to_csv('../Modelo_ER/source_region_electricity.csv', index=False)